In [1]:
import os 
import numpy as np
import pandas as pd
from datetime import date, timedelta


import matplotlib.pyplot as plt
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Load Data

In [2]:
path = '../data/processed'
df = pd.read_pickle(os.path.join(path,'transactions.pkl'))
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,2,663713001,0.0508,2
1,2018-09-20,2,541518023,0.0305,2
2,2018-09-20,7,505221004,0.0152,2
3,2018-09-20,7,685687003,0.0169,2
4,2018-09-20,7,685687004,0.0169,2


## Target Matrix
- cus_id
- week_start
- week_end
- purchased_product_list
- total_price

In [17]:
print('First transaction: ', df.t_dat.min())
print('Last transaction: ',df.t_dat.max())

First transaction:  2018-09-20 00:00:00
Last transaction:  2020-09-22 00:00:00


In [18]:
df['week'] = df['t_dat'].dt.isocalendar().week 
df['year'] = df['t_dat'].dt.year
df['week_start'] = df['t_dat'].dt.to_period('W').apply(lambda r: r.start_time)
df['week_end'] = df['t_dat'].dt.to_period('W').apply(lambda r: r.end_time).dt.normalize()
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,year,week_start,week_end
0,2018-09-20,2,0663713001,0.0508,2,38,2018,2018-09-17,2018-09-23
1,2018-09-20,2,0541518023,0.0305,2,38,2018,2018-09-17,2018-09-23
2,2018-09-20,7,0505221004,0.0152,2,38,2018,2018-09-17,2018-09-23
3,2018-09-20,7,0685687003,0.0169,2,38,2018,2018-09-17,2018-09-23
4,2018-09-20,7,0685687004,0.0169,2,38,2018,2018-09-17,2018-09-23


### purchased_product_list

In [36]:
weekly_purchased = pd.DataFrame(df.groupby(['customer_id','week','year'])['article_id'].apply(lambda x: list(set(x))))\
                        .reset_index()\
                        .rename(columns={'article_id':'weekly_purchased_products'})
# weekly_purchased['weekly_purchased_products'] = weekly_purchased['weekly_purchased_products'].apply(lambda x: list(set(x)))
weekly_purchased.head()

,customer_id,week,year,weekly_purchased_products
0,0,12,2020,"[0841260003, 0890498002, 0887593002, 085941601..."
1,0,18,2019,[0697138006]
2,0,21,2019,[0568601006]
3,0,30,2019,"[0745232001, 0607642008]"
4,0,36,2020,[0568601043]


### total_price

In [37]:
total_price = df.groupby(['customer_id','week','year'])\
                .agg({'article_id':'count','price':sum})\
                .reset_index()\
                .rename(columns={'article_id':'total_articles','price':'total_amount'})
total_price.head()

,customer_id,week,year,total_articles,total_amount
0,0,12,2020,5,0.0936
1,0,18,2019,1,0.0102
2,0,21,2019,2,0.1017
3,0,30,2019,2,0.0339
4,0,36,2020,1,0.0508


### Final DataFrame

In [38]:
final_df = total_price.merge(weekly_purchased,on=['customer_id','week','year'],how='left')
final_df.head()

,customer_id,week,year,total_articles,total_amount,weekly_purchased_products
0,0,12,2020,5,0.0936,"[0841260003, 0890498002, 0887593002, 085941601..."
1,0,18,2019,1,0.0102,[0697138006]
2,0,21,2019,2,0.1017,[0568601006]
3,0,30,2019,2,0.0339,"[0745232001, 0607642008]"
4,0,36,2020,1,0.0508,[0568601043]


In [39]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8090355 entries, 0 to 8090354
Data columns (total 6 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   customer_id                int64  
 1   week                       int64  
 2   year                       int64  
 3   total_articles             int64  
 4   total_amount               float32
 5   weekly_purchased_products  object 
dtypes: float32(1), int64(4), object(1)
memory usage: 401.2+ MB


In [40]:
final_df.to_pickle(os.path.join(path,'weekly_target.pkl'))

## Sampling Data
- Exclude cold-start customers and one-transaction customers
- Exclude customers with outlier transaction


In [3]:
path = '../data/processed'
trans = pd.read_pickle(os.path.join(path,'transactions.pkl'))
customers = pd.read_pickle(os.path.join(path,'customers.pkl'))
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,0,0.0000,0.0000,ACTIVE,NONE,49.0000,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,1,0.0000,0.0000,ACTIVE,NONE,25.0000,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,2,0.0000,0.0000,ACTIVE,NONE,24.0000,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,3,0.0000,0.0000,ACTIVE,NONE,54.0000,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,4,1.0000,1.0000,ACTIVE,Regularly,52.0000,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [11]:
temp = trans.groupby('customer_id').agg({'t_dat':'nunique','article_id':'count'}).reset_index()
temp.describe(percentiles=[0.1,0.25,0.5,0.75,0.9,0.99])

,customer_id,t_dat,article_id
count,1362281.0000,1362281.0000,1362281.0000
mean,685950.8047,6.6654,23.3346
std,396054.1843,9.7070,39.2423
min,0.0000,1.0000,1.0000
10%,137164.0000,1.0000,2.0000
25%,342966.0000,1.0000,3.0000
50%,685902.0000,3.0000,9.0000
75%,1028907.0000,8.0000,27.0000
90%,1234778.0000,17.0000,60.0000
99%,1358255.2000,45.0000,187.0000


In [18]:
exclude_list = temp['customer_id'][(temp.t_dat<=1)|(temp.article_id>180)]
print('Total customers: ',len(temp))
print('Total excluded customers: ',len(exclude_list))
print('Exclusion ratio: {:.2%}'.format(len(exclude_list)/len(temp)))

print('Remained customers: {}'.format(len(temp)-len(exclude_list)))

Total customers:  1362281
Total excluded customers:  462910
Exclusion ratio: 33.98%
Remained customers: 899371


### Random Sampling

In [48]:
from random import sample
random_list = temp['customer_id'][-temp.customer_id.isin(exclude_list)].sample(100000)
# random_list = sample(eligible_list,100000)
temp[temp.customer_id.isin(random_list)].describe()

,customer_id,t_dat,article_id
count,100000.0000,100000.0000,100000.0000
mean,687483.3983,8.7749,29.1674
std,396085.8096,8.8293,31.0446
min,12.0000,2.0000,2.0000
25%,345518.2500,3.0000,8.0000
50%,688379.5000,6.0000,18.0000
75%,1029583.0000,11.0000,38.0000
max,1371976.0000,116.0000,180.0000


In [49]:
print('First t_dat: ', trans[trans.customer_id.isin(random_list)]['t_dat'].min())
print('Last t_dat: ', trans[trans.customer_id.isin(random_list)]['t_dat'].max())
print('Total transactions: ', len(trans[trans.customer_id.isin(random_list)]))
target_cus = trans['customer_id'][(trans.customer_id.isin(random_list))&(trans.t_dat >= '2020-09-15')].unique()
print('Total target customers: {} ({:.2%})'.format(len(target_cus),len(target_cus)/len(random_list)))

First t_dat:  2018-09-20 00:00:00
Last t_dat:  2020-09-22 00:00:00
Total transactions:  2916736
Total target customers: 7170 (7.17%)


### Save Sampling

In [52]:
output = pd.DataFrame(target_cus,columns=['customer_id'])
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7170 entries, 0 to 7169
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   customer_id  7170 non-null   int64
dtypes: int64(1)
memory usage: 56.1 KB
